## Generate FBM data
- BehavePlus v.6 (BP6)
- FBP
- KITRAL

In [18]:
import os, itertools, subprocess
import pandas as pd
import numpy as np

In [19]:
os.chdir('/Users/minho/Desktop/Cell2FireML/Github/notebooks')

#### BP6

In [ ]:
# Import csv file with input features to calculate Behave Plus
# df = pd.read_csv('/Users/minho/Desktop/Cell2Fire/behaveplus/behaveplus_csv.csv')

# Iterate through different dataset combinations to create csv to push into Cell2FireC
# fuel model (40) x ws (6) x wd (4) x scenario (4) x slope (6)
a = [[101, 102, 103, 104, 105, 106, 107, 108, 109, 121, 122, 123, 124, 141, 142, 143, 144, 145, 146, 147, 148, 149, 161, 162, 163, 164, 165, 181, 182, 183, 184, 185, 186, 187, 188, 189,201, 202, 203, 204], 
     [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90], # WS
     # [0, 90, 180, 270], # WD
     [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85], # Slope
     [1,2,3,4]]

aa = list(itertools.product(*a))

adf = pd.DataFrame(np.asarray(aa), columns = ['nftype','ws','slope','scenario'])

# Moisture Content
mc_dict = {1: [3, 4, 5, 30, 60],
           2: [6, 7, 8, 60, 90],
           3: [9, 10, 11, 90, 120],
           4: [12, 13, 14, 120, 150]}

mc = pd.DataFrame(adf['scenario'].map(mc_dict).tolist(), columns = ['mc1','mc10','mc100','mcWoody','mcHerb'] )

# Merge df together
adf = adf.drop('scenario', axis=1)
adf = pd.concat([adf, mc], axis=1)

# Save
# adf.to_csv('behaveplus_training_data.csv')

#### FBP (CANADA)

In [20]:
# Data generator
def data_generator(FT=['C1','C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'D1', 'D2', 'S1', 'S2', 'S3', 'O1a', 'O1b', 'M1', 'M2', 'M3', 'M4'],
                   WS=[0,100,1],
                   FFMC=[0,101,1],
                   BUI=[0,99,1],
                   ps=[0,100,1], 
                   saz=[0,360,1], 
                   specific_values=False):
    
    # Constants
    # Lat/lon (Arbitrary for now)
    lat = 51.621244
    lon = -115.608378
    time = 20
    
    # GFL dictionary
    GFLD = {"C1": 0.75, "C2": 0.8, "C3": 1.15, "C4": 1.2, "C5":1.2, "C6":1.2, "C7":1.2, 
            "D1": np.nan, "D2": np.nan, 
            "S1":np.nan, "S2": np.nan, "S3": np.nan, 
            "O1a":0.35, "O1b":0.35, 
            "M1": np.nan, "M2": np.nan, "M3":np.nan, "M4":np.nan, "NF":np.nan,
            "M1_5": 0.1, "M1_10": 0.2,  "M1_15": 0.3, "M1_20": 0.4, "M1_25": 0.5, "M1_30": 0.6, 
            "M1_35": 0.7, "M1_40": 0.8, "M1_45": 0.8, "M1_50": 0.8, "M1_55": 0.8, "M1_60": 0.8, 
            "M1_65": 1.0, "M1_70": 1.0, "M1_75": 1.0, "M1_80": 1.0, "M1_85": 1.0, "M1_90": 1.0, "M1_95": 1.0}
    
    # PDF dictionary
    PDFD ={"M3_5": 5,"M3_10": 10,"M3_15": 15,"M3_20": 20,"M3_25": 25,"M3_30": 30,"M3_35": 35,"M3_40": 40,"M3_45": 45,"M3_50": 50,
           "M3_55": 55,"M3_60": 60,"M3_65": 65,"M3_70": 70,"M3_75": 75,"M3_80": 80,"M3_85": 85,"M3_90": 90,"M3_95": 95,"M4_5": 5,
           "M4_10": 10,"M4_15": 15,"M4_20": 20,"M4_25": 25,"M4_30": 30,"M4_35": 35,"M4_40": 40,"M4_45": 45,"M4_50": 50,"M4_55": 55,
           "M4_60": 60,"M4_65": 65,"M4_70": 70,"M4_75": 75,"M4_80": 80,"M4_85": 85,"M4_90": 90,"M4_95": 95,"M3M4_5": 5,"M3M4_10": 10,
           "M3M4_15": 15,"M3M4_20": 20,"M3M4_25": 25,"M3M4_30": 30,"M3M4_35": 35,"M3M4_40": 40,"M3M4_45": 45,"M3M4_50": 50,"M3M4_55": 55,
           "M3M4_60": 60,"M3M4_65": 65,"M3M4_70": 70,"M3M4_75": 75,"M3M4_80": 80,"M3M4_85": 85,"M3M4_90": 90,"M3M4_95": 95}
    
    # PCD dictionary
    PCD = {"M1_5":5,"M1_10":10,"M1_15":15,"M1_20":20,"M1_25":25,"M1_30":30,"M1_35":35,"M1_40":40,"M1_45":45,
           "M1_50":50,"M1_55":55,"M1_60":60,"M1_65":65,"M1_70":70,"M1_75":75,"M1_80":80,"M1_85":85,"M1_90":90,
           "M1_95":95,"M2_5":5,"M2_10":10,"M2_15":15,"M2_20":20,"M2_25":25,"M2_30":30,"M2_35":35,"M2_40":40,
           "M2_45":45,"M2_50":50,"M2_55":55,"M2_60":60,"M2_65":65,"M2_70":70,"M2_75":75,"M2_80":80,"M2_85":85,
           "M2_90":90,"M2_95":95,"M1M2_5":5,"M1M2_10":10,"M1M2_15":15,"M1M2_20":20,"M1M2_25":25,"M1M2_30":30,
           "M1M2_35":35,"M1M2_40":40,"M1M2_45":45,"M1M2_50":50,"M1M2_55":55,"M1M2_60":60,"M1M2_65":65,"M1M2_70":70,
           "M1M2_75":75,"M1M2_80":80,"M1M2_85":85,"M1M2_90":90,"M1M2_95":95}
    
    # Create empty dataframe
    cols = ['fueltype', 'mon', 'jd', 'M', 'jd_min',	'lat', 'lon', 'elev', 'ffmc', 'ws',	'waz', 'bui', 'ps',	'saz', 'pc', 'pdf',	'gfl', 'cur', 'time', 'pattern']
    df_data = pd.DataFrame(columns=cols)
    
    # Mix
    print('Creating combinations...')
    if specific_values is False:
        ws = np.arange(WS[0], WS[1] + 1, WS[2])
        ffmc = np.arange(FFMC[0], FFMC[1] + FFMC[2], FFMC[2])
        bui = np.arange(BUI[0], BUI[1] + BUI[2], BUI[2])
        pss = np.arange(ps[0], ps[1] + ps[2], ps[2])
        sazs = np.arange(saz[0], saz[1] + saz[2], saz[2])
    else:
        ws = WS
        ffmc = FFMC
        bui = BUI
        pss = ps
        sazs = saz
    combinations = np.stack(np.meshgrid(FT, ws, ffmc, bui, pss, sazs), -1).reshape(-1, 6)    
    _aux = pd.DataFrame(combinations)
    _aux.columns = ['fueltype', 'ws', 'ffmc', 'bui', 'ps', 'saz']
    
    # Populating gfl
    for ft in GFLD.keys():
        _aux.loc[_aux['fueltype'] == ft, 'gfl'] = GFLD[ft]
    
    # Populate
    print('Populating final df...')
    df_data  = pd.concat([df_data, _aux], axis=0)
    df_data['lat'] = lat
    df_data['lon'] = lon
    df_data['time'] = time
    
    # Return generated dataset
    return df_data

In [46]:
# Generate the data
df_datagen = data_generator(FT=['C1','C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'D1', 'D2', 'S1', 'S2', 'S3', 'O1a', 'O1b', 'M1', 'M2', 'M3', 'M4'],
                            WS=np.arange(0,80,5),
                            FFMC=[40, 85, 89, 91, 95],
                            BUI=[10, 25, 45, 65, 99],
                            ps=[0, 25, 50, 75, 100],
                            saz=[0, 45, 90, 135, 180],
                            specific_values=True)
print('Dims:', df_datagen.shape)

instance_name = 'DataGenerated_Small'
df_datagen.to_csv('DataGenerated_Small' + '.csv', index=False)
df_datagen.head()

Creating combinations...
Populating final df...
Dims: (180000, 20)


,fueltype,mon,jd,M,jd_min,lat,lon,elev,ffmc,ws,waz,bui,ps,saz,pc,pdf,gfl,cur,time,pattern
0,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,0,NaN,NaN,0.75,NaN,20,NaN
1,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,45,NaN,NaN,0.75,NaN,20,NaN
2,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,90,NaN,NaN,0.75,NaN,20,NaN
3,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,135,NaN,NaN,0.75,NaN,20,NaN
4,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,180,NaN,NaN,0.75,NaN,20,NaN


In [47]:
# Dict class [dict to object]
class Dict2Class(object):
    def __init__(self, my_dict):
        for key in my_dict:
            setattr(self, key, my_dict[key])

# Run FBP given input data to obtain HROS, BROS, and FROS
def run_fbp(fbp_path, args):
    execArray = [
        os.path.join(fbp_path, 'FBPDataGenerator'),         # Executable for FBP data generator (C++)
        '--input-data', args['InFolder'] + args['instance'],
        # '--nsims', args['NSims'],
        '--verbose' if args['verbose'] else '',
    ]
    
    # Verbose
    if args['verbose']:
        print('ExecArray:', execArray)
    
    # Output Log
    if args['OutFolder'] is not None:
        if os.path.isdir(args['OutFolder']) is False:
            os.makedirs(args['OutFolder'])
    LogName = os.path.join(args['OutFolder'], 'FBPDataGenerator_Output.csv')
        
    # Perform the call 
    with open(LogName, 'w') as output:
        proc = subprocess.Popen(execArray, stdout=output)
        proc.communicate()
    return_code = proc.wait()
    if return_code != 0:
        raise RunTimeError(f'C++ returned {return_code}. \nTry looking at {LogName}.')
        
    # End data generator execution
    print('End of FBP DataGenerator execution...')
    return execArray

In [52]:
# Paths
FBP_DATA_GEN_PATH = '/Users/minho/Desktop/Cell2FireML/Github/notebooks'

# Arguments
args = {
    'InFolder': os.path.join(FBP_DATA_GEN_PATH, instance_name),
    'OutFolder': os.path.join(FBP_DATA_GEN_PATH, instance_name),
    'NSims':1,
    'verbose': False,
    'instance': instance_name
}

# Dict to object
args_obj = Dict2Class(args)

# Run Cell2Fire with FBP
execArray = run_fbp(fbp_path=FBP_DATA_GEN_PATH, args=args)

In [50]:
# Read
df_ros = pd.read_csv('DataGenerated_Small.csv', low_memory=False)
print('Dims:', df_ros.shape)
df_ros.head()

# df_train_data.to_csv('training_data_small_08202023.csv', index=False)

Dims: (180000, 20)


,fueltype,mon,jd,M,jd_min,lat,lon,elev,ffmc,ws,waz,bui,ps,saz,pc,pdf,gfl,cur,time,pattern
0,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,0,NaN,NaN,0.75,NaN,20,NaN
1,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,45,NaN,NaN,0.75,NaN,20,NaN
2,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,90,NaN,NaN,0.75,NaN,20,NaN
3,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,135,NaN,NaN,0.75,NaN,20,NaN
4,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,180,NaN,NaN,0.75,NaN,20,NaN


In [17]:
# Merge inputs/outputs
df_train_data = pd.concat([df_datagen, df_ros], axis=1)
print('Original Dims:', df_train_data.shape)

# # Take away NAN values for ROS (useless)
# df_train_data.dropna(subset=['HROS'], inplace=True)
# df_train_data.dropna(subset=['FROS'], inplace=True)
# df_train_data.dropna(subset=['BROS'], inplace=True)
# print('Dims:', df_train_data.shape)
# df_train_data.head()

Original Dims: (180000, 40)


#### KITRAL (CHILE)
- Contenido Humedad: 0.5% steps  -> 20 + 1 vals
- Humedad: 1% steps -> 30 + 1 vals
- Viento: 0.5 km/hr steps -> 60 + 1 vals
- Pendiente: 0.5% steps -> 60 + 1 vals

FCH ("factor contenido humedad"), FMC ("factor de propagación"), FP ("factor pendiente") and FV ("factor viento")
- FP and FV represents the effect of slope and wind speed.

In [ ]:
# Constants
speed = [0.0188800,0.0160270,0.0102350,0.0086900,0.0010090,0.0076030,0.0081470,0.0016720,0.0048860,0.0103210,0.0092340,0.0017870,0.0043420,0.0022490,0.0014410,0.0009790,0.0015560,0.0023650,0.0131740,0.0059730,0.0024810,0.0027120,0.0065160,0.0032550,0.0025960,0.0097770,0.0054290,0.0037990,0.0013250,0.0021340,0.0019030]
carga = [0.684,0.527,0.918,0.617,0.649,2.923,1.910,3.308,1.383,3.029,3.529,3.189,1.903,2.624,2.310,3.544,2.164,1.954,0.838,3.019,3.333,3.249,4.087,3.714,4.063,0.905,3.164,2.742,2.464,8.250,7.125]
heat = [3928,3928,3928,3928,3800,4693,4693,4572,4572,5000,5087,4500,4500,4600,4550,4452,4452,4452,4399,4870,4870,4870,4870,4870,4870,4372,4816,4816,4684,4746,4652]

speed_dict = dict(zip(np.arange(1,32,1),speed))
carga_dict = dict(zip(np.arange(1,32,1),carga))
heat_dict = dict(zip(np.arange(1,32,1),heat))

# Variables
fm = np.arange(1,32,1).tolist()
ch = np.arange(0,21,5).tolist()
# humidity = np.arange(0,32,2).tolist()
wind = np.arange(0,61,5).tolist()
slope = np.arange(0,61,5).tolist()

In [ ]:
# Iterate through all lists
a = [fm, ch, wind, slope]
aa = list(itertools.product(*a))

# Convert lists to DF
adf = pd.DataFrame(np.asarray(aa))
adf.columns = ['nftype','ch','wind','slope']

# Map constants based on nftype with variables
adf['speed'] = adf['nftype'].map(speed_dict)
adf['carga'] = adf['nftype'].map(carga_dict)
adf['heat'] = adf['nftype'].map(heat_dict)
# adf['fmc'] = adf.speed

# Variables
adf['fch'] = [(389.1624-14.3*ch+0.02*ch**2)/(3.559+1.6615*ch+2.6239*ch**2) for ch in adf.ch]
adf['fp'] = [1+0.023322*slope+0.00013585*slope**2 for slope in adf.slope]
adf['fv'] = [1+0.51218*wind-0.007151*wind**2 for wind in adf.wind]

# Outputs
l1 = 2.233/1.411
l2 = -0.01031/0.01745
adf['HROS'] = adf.speed * adf.fch * (adf.fp + adf.fv)
adf['lb'] = [1.0 + pow(l1 * np.exp(-l2 * ws) - l1, 2.0) for ws in adf.wind]
adf['hb'] = [(lb + np.sqrt(pow(lb,2) - 1.0)) /  (lb - np.sqrt(pow(lb, 2) - 1.0)) for lb in adf.lb]

adf['BROS'] = adf.HROS/adf.hb
adf['FROS'] = (adf.HROS + adf.BROS) / ( adf.lb * 2.0)

adf = adf[['nftype', 'speed', 'carga', 'heat', 'ch','wind','slope','fch','fp','fv','HROS','BROS','FROS']]
# adf.to_csv("data/kitral_training_3ros_full.csv",index=None)